In [1]:
# Collecting data
import polars as pl

df = pl.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
)
df2 = pl.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
)

df2 = df2.rename({"Airport_fee": "airport_fee"})

df2.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2023-02-01 00:32:53,2023-02-01 00:34:34,2,0.3,1,"""N""",142,163,2,4.4,3.5,0.5,0.0,0.0,1.0,9.4,2.5,0.0
2,2023-02-01 00:35:16,2023-02-01 00:35:30,1,0.0,1,"""N""",71,71,4,-3.0,-1.0,-0.5,0.0,0.0,-1.0,-5.5,0.0,0.0
2,2023-02-01 00:35:16,2023-02-01 00:35:30,1,0.0,1,"""N""",71,71,4,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0
1,2023-02-01 00:29:33,2023-02-01 01:01:38,0,18.8,1,"""N""",132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
2,2023-02-01 00:12:28,2023-02-01 00:25:46,1,3.22,1,"""N""",161,145,1,17.0,1.0,0.5,3.3,0.0,1.0,25.3,2.5,0.0


In [2]:
# Read the data for January. How many columns are there?
print(df.shape[1])

19


In [3]:
# What's the standard deviation of the trips duration in January?
print(
    df.select(
        (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
        .dt.total_minutes()
        .std()
        .alias("trip_time_in_minutes_std")
    ).item(0, 0)
)

42.59449915808726


In [4]:
within_1_hour = df.filter(
    (
        (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
        .dt.total_minutes()
        .lt(60)
    )
    & (
        (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
        .dt.total_minutes()
        .gt(0)
    )
)

within_1_hour_test = within_1_hour.select(
    pl.col("tpep_pickup_datetime").alias("pickup_time"),
    pl.col("tpep_dropoff_datetime").alias("dropoff_time"),
    (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")).alias(
        "trip_time"
    ),
)
a = within_1_hour_test.select(pl.col("trip_time").count().alias("count_trip_time")).item(
    0, 0
)
b = df.shape[0]

d = a / b * 100
print(f"{d:.2f}%")

98.12%


In [5]:
# Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it

# What's the dimensionality of this matrix (number of columns)?


from sklearn.feature_extraction import DictVectorizer


hot_df = within_1_hour.select(
    pl.col("PULocationID").cast(pl.String).alias("PULocationID"),
    pl.col("DOLocationID").cast(pl.String).alias("DOLocationID"),
).to_pandas()

dict_list = hot_df.to_dict(orient="records")

vec = DictVectorizer()
feature_matrix = vec.fit_transform(dict_list)

print(feature_matrix.shape[1])

515


In [6]:
# Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters
# Calculate the RMSE of the model on the training data

# What's the RMSE on train?

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


X_train = feature_matrix
y_train = (
    within_1_hour.select(
        ((pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")).dt.total_minutes()).alias("trip_time")
    )
    .to_pandas()["trip_time"]
    .values
)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.654040673597321

In [7]:
within_1_hour2 = df2.filter(
    (
        (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
        .dt.total_minutes()
        .lt(60)
    )
    & (
        (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
        .dt.total_minutes()
        .gt(0)
    )
)

hot_df2 = within_1_hour2.select(
    pl.col("PULocationID").cast(pl.String).alias("PULocationID"),
    pl.col("DOLocationID").cast(pl.String).alias("DOLocationID"),
).to_pandas()

dict_list2 = hot_df2.to_dict(orient="records")

X_val = vec.transform(dict_list2)

y_pred = lr.predict(X_val)

y_val = (
    within_1_hour2.select(
        (
            (
                pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")
            ).dt.total_minutes()
        ).alias("trip_time")
    )
    .to_pandas()["trip_time"]
    .values
)

mean_squared_error(y_val, y_pred, squared=False)

/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.816190088188503